# Synteza Tacotron2

**Właściciel tego notatnika: Tadejro**

Link do oryginalnego notatnika: https://colab.research.google.com/drive/1HLtvXS1X1mChiXHM02myLz3SUhzJ4bqx

Link do źródła w repozytorium: https://github.com/8tm/mekatron-files/blob/master/colab_notepads/tacotron2/tacotron2_synthesis.ipynb

Połączenie wszystkiego, co do tej pory powstało na Mekatronie. Podziękowania dla:
- Meka
- Adam is cool and stuff – za orginalną wersję tacotronową
- Heroba – za zrobienie wersji z formularzami (Link: https://colab.research.google.com/drive/1FXQRoCRl4NW1DY77HiRcIySjrjS-wfSb)
- Patryka – za funkcję "zagraniczny_głos" i naprawienie pobierania z dysków Google
- Vojaka – za oryginalną wersję tego notatnika
- Patryk025 - za poprawki w hifi-gan i jeszcze kilka rzeczy

Obecnie notatnik posiada drobny błąd – przy pierwszej syntezie po uruchomieniu tego Colaba skrypt "zagraniczny_głos" nie załapuje, więc trzeba jeszcze raz wygenerować. Wszystkie kolejne generacje są już OK. Jak ktoś zna się na programowaniu, to może pomóc mi w naprawie tego błędu.

# Krok 1: Sprawdzenie karty graficznej

In [ ]:
#@title # Informacje o karcie graficznej i wersję pythona
_, card = !nvidia-smi --query-gpu=gpu_name --format='csv'
print(card)
!python -V

In [ ]:
#@title #Instalacja paczek
%%sh
echo "Packages for Python 3.10.12"
# Packages required for training and for synthesis
pip install -q \
    gdown==4.7.1 \
    unidecode==1.3.6 \
    tensorboardX==2.6.1 \
    pyunpack==0.3 \
    patool==1.12 \
    pynvml==11.5.0 \
    librosa==0.10.0-2 \
    pydub==0.25.1 \
    ffmpeg==1.4


# Krok 2: Import bibliotek, załadowanie własnych funkcji

In [ ]:
#@title
#Instalacja Tacotrona i Waveglow

import glob
import locale
import os
import sys
import time
import torch

import gdown
import matplotlib
import matplotlib.pylab as plt
import numpy as np
import IPython.display as ipd
import scipy.io.wavfile
from pydub import AudioSegment
from scipy.io.wavfile import write


def git_clone(url):
    project_name = os.path.splitext(os.path.basename(url))[0]
    if not os.path.exists(project_name):
        !git clone -q --recursive {url}
    return project_name


project_name = git_clone('https://github.com/8tm/mekatron2.git')

sys.path.append(os.path.join(project_name, 'waveglow/'))
sys.path.append(project_name)

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT
from audio_processing import griffin_lim
from text import text_to_sequence
from denoiser import Denoiser


def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

def plot_data(data, figsize=(9, 4)):
    %matplotlib inline
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom',
                       interpolation='none', cmap='inferno')
    fig.canvas.draw()
    plt.show()

def ARPA(text):
    out = ''
    for word_ in text.split(" "):
        word=word_; end_chars = ''
        while any(elem in word for elem in r"!?,.;") and len(word) > 1:
            if word[-1] == '!': end_chars = '!' + end_chars; word = word[:-1]
            if word[-1] == '?': end_chars = '?' + end_chars; word = word[:-1]
            if word[-1] == ',': end_chars = ',' + end_chars; word = word[:-1]
            if word[-1] == '.': end_chars = '.' + end_chars; word = word[:-1]
            if word[-1] == ';': end_chars = ';' + end_chars; word = word[:-1]
            else: break
        try: word_arpa = thisdict[word.upper()]
        except: word_arpa = ''
        if len(word_arpa)!=0: word = "{" + str(word_arpa) + "}"
        out = (out + " " + word + end_chars).strip()
    if out[-1] != ";": out = out + ";"
    return out

def download_from_google_drive(link, path, quiet=False):
    gdown.download(
        f'https://drive.google.com/uc?id={link.strip()}', path, quiet=quiet
    )

locale.getpreferredencoding = getpreferredencoding


----------------------------------------------------------------

# Krok 3: Ustawienie modelu

In [ ]:
#@title
#@markdown Jeżeli będzie problem z uprawnieniami do pliku, a są ustawione jak należy, to trzeba trochę poczekać bądź sprawdzić nazwę.
#@markdown ### Wymuszone pobieranie modeli
#@markdown Po prostu ponownie pobierze zaznaczone modele nawet, jeśli wcześniej były pobrane.
pobierz_model_tacotron2 = True #@param{type:'boolean'}
pobierz_model_waveglow = True #@param{type:'boolean'}

#@markdown ### ID modeli (pobiera je się z linku)
#@markdown ID modelu tacotron2:
tacotron2_link = "1df5KQ0WwxllwLsEbp7vdzuqHshhuM5JU" #@param{type:"string"}

#@markdown ID modelu waveglow:
waveglow_link = "17xuBnKr6gtGfR21Hmsgx_rk8kKPrEWrn" #@param{type:"string"}
tacotron2_pretrained_model = 'tacotron2'

if not os.path.exists(tacotron2_pretrained_model) or pobierz_model_tacotron2:
    # Pobieranie modelu Tacotron2
    download_from_google_drive(tacotron2_link, tacotron2_pretrained_model)

waveglow_pretrained_model = 'waveglow'
if not os.path.exists(waveglow_pretrained_model) or pobierz_model_waveglow:
    # Pobieranie modelu Waveglow
    download_from_google_drive(waveglow_link, waveglow_pretrained_model)


# Krok 4: Kod syntezy, ukryty by nie zapychał miejsca

In [ ]:
#@title
#Wstępne przygotowania Tacotrona i Waveglow
%matplotlib inline

# Download files from Google drive
download_from_google_drive('1uciIczSNxzjhWZEzJ_gB2pzLGrtXSQHc', 'merged.dict.txt')
download_from_google_drive('1qpgI41wNXFcH-iKq1Y42JlBC9j0je8PW', 'pretrained')
download_from_google_drive('1pAB2kQunkDuv6W5fcJiQ0CY8xcJKB22e', 'config.json')

thisdict = {}

for line in reversed((open('merged.dict.txt', "r").read()).splitlines()):
    thisdict[(line.split(" ",1))[0]] = (line.split(" ",1))[1].strip()

#torch.set_grad_enabled(False)

# initialize Tacotron2 with the pretrained model
hparams = create_hparams()

git_clone('https://github.com/patryk025/hifi-gan')

try:
    os.mkdir("test_files")
except FileExistsError:
    pass

----------------------------------------------------------------

# Krok 5: Ustawienie parametrów syntezy



In [ ]:
#@title
#@markdown **Częstotliwość próbkowania**. Większa częstotliwość próbkowania zwiększa prędkość głosu i jego ton. (domyślnie jest 22050). Lepiej nie zmieniać bazowej wartości.
hparams.sampling_rate = 22050 #@param{type:'number', min:'0', max:'384000'}
#@markdown **Maksymalna ilość iteracji dekodera** wpływa na długość wygenerowanego głosu.

#@markdown Można zwiększyć, by syntezator mógł wygenerować dłuższy głos (domyślnie 1000)
hparams.max_decoder_steps = 5000# @param{type:'number',min:1}
hparams.gate_threshold = 0.1 # Model must be 90% sure the clip is over before ending generation (the higher this number is, the more likely that the AI will keep generating until it reaches the Max Decoder Steps)
model = Tacotron2(hparams)
model.load_state_dict(torch.load(tacotron2_pretrained_model)['state_dict'])
_ = model.cuda().eval().half()

# Załaduj Waveglow
waveglow = torch.load(waveglow_pretrained_model)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

----------------------------------------------------------------

# Krok 6: Synteza

Od teraz dla zaoszczędzenia miejsca przykładowe teksty można wybrać z wysuwanego menu w inpucie *text*.

In [ ]:
#@title
#@markdown **Tekst do odczytania**
text = "Powierzy\u0142em swoje dziadki dla Antosi i jej Matki. Dzi\u015B Antosia powidzia\u0142a, \u017Ce \u017Cadnego dzieciaka nie bra\u0142a wi\u0119c i nie od da\u0142a. Co za wredna pa\u0142a." #@param ["Nam strzelać nie kazano. Wstąpiłem na działo. I spojrzałem na pole. dwieście armat grzmiało. Artyleryji ruskiej ciągną się szeregi. Prosto, długo, daleko, jako morza brzegi.", "Wasali przytłoczyło bogactwo i ogrom lądu i nie byli w stanie znaleźć miejsca na wybudowanie ich nowej stolicy.", "Korzystanie z dwóch broni oznacza, że nie masz żadnej ochrony i nie możesz blokować. Ale jeśli uderzysz dwiema broniami naraz, wróg może już nie mieć szansy odpowiedzieć.", "Z gwiazd obłąkanych, z włosów czesanych, iskry padają, jak z polskiej szabli. widzą je diabli, odpowiadają błyskami. Lecą. świst piór buki ugina.", "Jesteś dla mnie jak starszy brat. Nigdy nie miałem starszego brata, ani nawet młodszego. Moi rodzice mnie nie kochali, ale to musiało być dla nich bardzo trudne.", "Wiesz... Zabawne jest to, że często jest tak, że najbardziej niebezpieczni okazują się właśnie ci, którzy działają dla wyższych celów.", "Niniejszym, w imieniu Aldmerskiego Dominium, na mocy konkordatu bieli i złota, aresztuję meka za wiarę w talosa.", "Przed użyciem zapoznaj się z treścią ulotki dołączonej do opakowania bądź skonsultuj się z lekarzem lub farmaceutą, gdyż każdy lek niewłaściwie stosowany zagraża Twojemu życiu lub zdrowiu.", "Elektrody otulone są metalowymi prętami otoczonymi sprasowaną otuliną. Dobierane są głównie w zależności od składu chemicznego, właściwości i gabarytów materiałów łączonych, ale także spodziewanej wytrzymałości złącza, rodzaju źródła prądu czy pozycji spawania."] {allow-input: true}

zagraniczny_głos = False #@param{type:"boolean"}
denoise_strength =  0.06 #@param{type:"number"}
equalize = True #@param{type:"boolean"}
gan = True #@param{type:"boolean"}
_sigma = 1
speed_multiplier = 1
raw_input = False


polish_letter_pronunciation = [
    ["ch", "h"], ["Ch", "H"], ["ó", "u"], ["x", "ks"], ["v", "w"], ["rz", "ż"], ["Rz", "Ż"], ["ą ", "om "], ["ą, ", "om, "], ["ą. ", "om. "], ["ą? ", "om? "], ["ą! ", "om! "], ["ą.", "om."], ["ąb", "omb"], ["ąc", "onc"], ["ąć", "onć"], ["ąd", "ond"], ["ąf", "onf"], ["ąg", "ong"], ["ąh", "onh"], ["ąj", "onj"], ["ąk", "onk"], ["ąl", "oln"], ["ął", "oł"], ["ąm", "om"], ["ąn", "ołn"], ["ąp", "omp"], ["ąs", "ons"], ["ąś", "onś"], ["ąt", "ont"], ["ąw", "omw"], ["ąz", "onz"], ["ąż", "onż"], ["ąź", "onź"], ["ę ", "e "], ["ęb", "enb"], ["ęc", "enc"], ["ęć", "enć"], ["ęd", "end"], ["ęg", "eng"], ["ęk", "enk"], ["ęl", "el"], ["ęł", "eł"], ["ęn", "en"], ["ęp", "emp"], ["ęś", "enś"], ["ęs", "ens"], ["ęt", "ent"], ["ęw", "emw"], ["ęz", "enz"], ["ęż", "enż"], ["ęź", "enź"], ["ę. ", "e. "], ["ę.", "e."], ["ę, ", "e, "], ["ę? ", "e? "], ["ę! ", "e! "]
]

files = glob.glob('test_files/*')
for f in files:
    os.remove(f)

if zagraniczny_głos:
    for letter, pronunciation in polish_letter_pronunciation:
        text = (text.replace(letter, pronunciation))

for line in text.split("\n"):
    if len(line) < 1:
        continue
    print(line)
    if raw_input:
        if line[-1] != ";":
            line=line+";"
    else:
        line = ARPA(line)
    print(line)

    sequence = np.array(text_to_sequence(text, ['basic_cleaners']))[None, :]
    sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()
    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
    with torch.no_grad():
        audio = waveglow.infer(mel_outputs_postnet, sigma=_sigma)
    audio_denoised = denoiser(audio, strength=denoise_strength)[:, 0]

    print("Zwykła wersja:")
    ipd.display(ipd.Audio(audio_denoised.cpu().numpy() , rate=hparams.sampling_rate * speed_multiplier))

    audio = ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate * speed_multiplier)
    audio = AudioSegment(audio.data[128:], frame_rate=hparams.sampling_rate * speed_multiplier, sample_width=2, channels=1)
    audio.export("test_files/testnt.wav", format="wav", bitrate="32k")

    !ffmpeg -loglevel quiet -i "test_files/testnt.wav" -ss 0.0000 -vcodec copy -acodec copy "test_files/test.wav"

    if equalize:
        !ffmpeg -loglevel quiet -y -i "test_files/test.wav" -ac 2 -af "aresample=44100:resampler=soxr:precision=15, equalizer=f=50:width_type=o:width=0.75:g=3.6, equalizer=f=3000:width_type=o:width=1.0:g=2.0, equalizer=f=10000:width_type=o:width=1.0:g=4.0" "test_EQ.wav"
        print("Wersja z EQ:")
        ipd.display(ipd.Audio("test_EQ.wav"))

    if gan:
        if equalize:
            print("\n---------Logi generacji GAN---------\n")
            !python hifi-gan/inference.py --checkpoint_file pretrained
            print("\n---------Koniec logów generacji GAN---------\n")
            print("Gan bez EQ:")
            ipd.display(ipd.Audio("generated_files/test_generated.wav"))
            !ffmpeg -loglevel quiet -y -i "generated_files/test_generated.wav" -ac 2 -af "aresample=44100:resampler=soxr:precision=15, equalizer=f=50:width_type=o:width=0.75:g=3.6, equalizer=f=3000:width_type=o:width=1.0:g=2.0, equalizer=f=10000:width_type=o:width=1.0:g=4.0" "generated_files/test_EQ.wav"
            print("Gan z EQ:")
            ipd.display(ipd.Audio("generated_files/test_EQ.wav"))
        else:
            print("\n---------Logi generacji GAN---------\n")
            !python hifi-gan/inference.py --checkpoint_file pretrained
            print("\n---------Koniec logów generacji GAN---------\n")
            print("Gan bez EQ:")
            ipd.display(ipd.Audio("generated_files/test_generated.wav"))


----------------------------------------------------------------